# CSE547 - Colab 8 
## Submodular Optimization



# Setup

Let's authenticate a Google Drive client to download the file we will be processing.

**Make sure to follow the interactive instructions.**

In [0]:
!pip install -U -q PyDrive

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
id='1QrBqLNd43Z2n2x0bxFI7ICgren_GqJMh'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('book_ratings.csv')

If you executed the cells above, you should be able to see the dataset we will use for this Colab under the "Files" tab on the left panel.

Next, we import some of the common libraries for our task.

In [0]:
import pandas as pd
import numpy as np
import time

import warnings
warnings.filterwarnings("ignore")

from heapq import heappop, heappush

# Your Task

If you run successfully the setup stage, you are ready to work with the **Book-Crossing (BX) dataset**. If you want to read more about it, check the [official website](http://www2.informatik.uni-freiburg.de/~cziegler/BX/) of the dataset, which contains concise schema description of the dataset, and the download page.

In this Colab, we will use the [Pandas API](https://pandas.pydata.org/docs/user_guide/index.html) to perform submodular optimiazation algorithms on a small subset of the BX-Book-Ratings dataset (20K users). 



In [5]:
data = pd.read_csv('book_ratings.csv')
data.head()

,userid,isbn,rating
0,276727,0446520802,0
1,276729,052165615X,3
2,276729,0521795028,6
3,276736,3257224281,8
4,276762,034544003X,0


Let's see the top 10 readers who have rated the most books, sorted in descending order. 

In [6]:
a = data.groupby('userid').isbn.nunique().sort_values(ascending=False)[:8]

np.array(a.index)

array([198711, 212898, 234623, 102967,  98741, 190925,  69697,  78783])

Your goal is to find the set of users who rated the most *unique* books by implementing the **greedy submodular optimization algorithm** and the **lazy greedy algorithm**. Note: A user may give a rating of 0 to a book. 


## Q1: Greedy Submodular Optimization Algorithm


We start by determining our objective function. Write a function that returns the number of unique books rated by the set of users A. 

In [7]:
def F(A=[198711,234623]):
  data_subset = data[data.userid.isin(A)]
  unique_ibsn = data_subset.isbn.nunique()
  return unique_ibsn 


F([198711, 212898, 234623])

14324

The greedy(k) function below provides a sketch for the greedy submodular optimization algorithm where the maximum cardinality is set to be k.  Your task is to complete the function and **report the set of users and the objective values (i.e. number of unique books) for k = 2 and 3.** (They may take a while to run). You are free to modify or implement your own code.

In [21]:
def greedy(k):
  # list of unique user id
  users = candidates = data.userid.unique()
  
  # initialize the list of candidate users
  greedy = []

  # initialize the time tracker
  time_k = 0

  # loop over the cardinality values
  for i in range(k):



    start_time = time.time()

    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # Find the user who contributes the most gain with respect to the 
    # objective value
    # Save this user in a variable `candidate_user` 
    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

    candidate_user = 0
    max_gain = 0
    for user in candidates:
      unique_isbn = F([user] + greedy)
      if unique_isbn > max_gain:
        candidate_user = user
        max_gain = unique_isbn

    # update the list of candidate users
    greedy = greedy + [candidate_user]

    end_time = time.time() - start_time
    time_k += end_time

    print('k = {}'.format(i+1))
    print("The set of users: {}".format(greedy))
    print("Objective value = {}".format(F(greedy)))
    print("Time taken = {} seconds".format(round(time_k, 5)))

    # -------------------------------------------
    # remove the chosen candidate from user list
    #-------------------------------------------
    candidates = candidates[candidates!=candidate_user]

greedy(8)

k = 1
The set of users: [198711]
Objective value = 7550
Time taken = 39.41379 seconds
k = 2
The set of users: [198711, 212898]
Objective value = 11963
Time taken = 118.07812 seconds
k = 3
The set of users: [198711, 212898, 234623]
Objective value = 14324
Time taken = 215.68252 seconds
k = 4
The set of users: [198711, 212898, 234623, 98741]
Objective value = 16377
Time taken = 332.06474 seconds
k = 5
The set of users: [198711, 212898, 234623, 98741, 190925]
Objective value = 18200
Time taken = 445.10414 seconds
k = 6
The set of users: [198711, 212898, 234623, 98741, 190925, 102967]
Objective value = 19813
Time taken = 574.05653 seconds
k = 7
The set of users: [198711, 212898, 234623, 98741, 190925, 102967, 23902]
Objective value = 21248
Time taken = 699.34253 seconds
k = 8
The set of users: [198711, 212898, 234623, 98741, 190925, 102967, 23902, 69697]
Objective value = 22647
Time taken = 835.34408 seconds


## Q2: Lazy Greedy Algorithm

Now, revise the code by implementing a lazy greedy optimization algorithm. **Report the set of users and the objective values for k = 7 and 8.**

*Hints:* (1) The results should be the same for all k between the two algorithms. (2) The objective value for k = 5 is 18200. (3) It may be useful, but not required, to use the [heapq](https://docs.python.org/2/library/heapq.html) algorithm in keeping the sorted array.









In [19]:
import heapq

def lazy(k): 

  # list of unique user id
  users = data.userid.unique()

  # initialize the list of candidate users
  lazy_greedy = []

  # ---------------------------------------------------------
  # You may want to initialize a list of marginal gains
  # --------------------------------------------------------
  marginal =[]
  for user in users:
    user = [user]
    marginal.append(F(user))

  # Create the sorted list of nodes and their marginal gain 
  Q = sorted(zip(users, marginal), key=lambda x: x[1], reverse=True)
  print(Q.pop(0))

  # initialize the time tracker
  time_k = 0
  
  # loop over the cardinality value
  for i in range(k):
    start_time = time.time()

    

    if i == 0:

      # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
      # Find the user who contributes the most gain with respect to the 
      # objective value, 
      # pop and save this user in a variable `candidate_user', 
      # keep the rest in a sorted array with repect to their marginal gains 
      # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
      candidate_user, spread = Q.pop(0)
      # Recalculate spread of top nodes





    else:  
      # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
      # This should include a while loop, where a comparison between the second
      # best candidate and the third best candidate is made. 
      # Once you find the next candidate, pop and save this user in `candidate_user' 
      # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
      check = False
      target = 0
      marginal = [0,0]
      while not check:
        
        # Recalculate spread of top nodes
        current = Q[0][0]
        
        # Evaluate the spread function and store the marginal gain in the list
        Q[0] = (current, F(lazy_greedy+[current]) - spread)
        
        # Re-sort the list
        Q = sorted(Q, key = lambda x: x[1], reverse = True)

        # Check if previous top node stayed on top after the sort
        check = (Q[0][0] == current)
      
      
      #print(spread)

    
    lazy_greedy.append(candidate_user)
    candidate_user = Q.pop(0)[0]
    spread = F(lazy_greedy)


    end_time = time.time() - start_time
    time_k += end_time 

    print('k = {}'.format(i+1))
    print("The set of users: {}".format(lazy_greedy))
    print("Objective value = {}".format(F(lazy_greedy)))
    print("Time taken = {} seconds".format(round(time_k, 5)))

  
lazy(8)
    

(198711, 7550)
k = 1
The set of users: [212898]
Objective value = 4785
Time taken = 0.00321 seconds
k = 2
The set of users: [212898, 234623]
Objective value = 7340
Time taken = 0.02012 seconds
k = 3
The set of users: [212898, 234623, 98741]
Objective value = 9511
Time taken = 0.03792 seconds
k = 4
The set of users: [212898, 234623, 98741, 102967]
Objective value = 11417
Time taken = 0.06342 seconds
k = 5
The set of users: [212898, 234623, 98741, 102967, 190925]
Objective value = 13269
Time taken = 0.08573 seconds
k = 6
The set of users: [212898, 234623, 98741, 102967, 190925, 69697]
Objective value = 14717
Time taken = 0.12781 seconds
k = 7
The set of users: [212898, 234623, 98741, 102967, 190925, 69697, 23902]
Objective value = 16159
Time taken = 0.19198 seconds
k = 8
The set of users: [212898, 234623, 98741, 102967, 190925, 69697, 23902, 76626]
Objective value = 17361
Time taken = 0.20661 seconds


In [15]:
sorted([198711, 212898, 198711, 212898, 198711, 212898, 198711])

[198711, 198711, 198711, 198711, 212898, 212898, 212898]

Once you obtained the desired results, **head over to Gradescope and submit your solution for this Colab**!